In [1]:
#!pip install tensorflow_transform

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
#import tensorflow_transform as ttf
import h5py as h

In [2]:
data=pd.read_csv('test_data_set_1.csv', header=0)

In [3]:
def split_data(X,y,split=0.1):
  temp =np.random.choice(list(range(len(y))), size=int(len(y)*split), replace = False)

  test_X = pd.DataFrame.from_dict({k: v[temp] for k,v in X.items()})
  #train_X = {k: np.delete(v,temp) for k,v in X.items()}
  train_X = pd.DataFrame.from_dict({k: v.drop(temp) for k,v in X.items()})

  test_y = y[temp]
  #train_y =np.delete(y, temp)
  train_y = y.drop(temp)

  return train_X, test_X, train_y, test_y


#dd=data.drop(['A','x'], axis = 1)
y = data['A']
X = data.drop(['A'], axis = 1)
train_X, test_X, train_y, test_y = split_data(X,y)

In [12]:
def create_model():
    input = tf.keras.Input(shape=(3))
    layer = tf.keras.layers.Dense(8, activation = 'relu')(input)
    output = tf.keras.layers.Dense(1, activation = 'linear')(layer)
    model = tf.keras.Model(inputs=[input], outputs=output, name="testmodel")
    model.compile(optimizer = tf.keras.optimizers.Adam(.02),
                        loss = tf.keras.losses.MeanSquaredError(), metrics=['mean_squared_error'])
    return model

In [13]:
model_test = create_model()
model_test.summary()

Model: "testmodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_6 (Dense)             (None, 8)                 32        
                                                                 
 dense_7 (Dense)             (None, 1)                 9         
                                                                 
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


In [14]:
model_test.fit(train_X,train_y, validation_data = (test_X, test_y), epochs = 50)

Epoch 1/50
2/2 [==============================] - 1s 200ms/step - loss: 8667.7725 - mean_squared_error: 8667.7725 - val_loss: 5181.2275 - val_mean_squared_error: 5181.2275
Epoch 2/50
2/2 [==============================] - 0s 49ms/step - loss: 5445.7212 - mean_squared_error: 5445.7212 - val_loss: 3117.3047 - val_mean_squared_error: 3117.3047
Epoch 3/50
2/2 [==============================] - 0s 69ms/step - loss: 3191.8542 - mean_squared_error: 3191.8542 - val_loss: 1696.6449 - val_mean_squared_error: 1696.6449
Epoch 4/50
2/2 [==============================] - 0s 55ms/step - loss: 1717.1595 - mean_squared_error: 1717.1595 - val_loss: 794.3433 - val_mean_squared_error: 794.3433
Epoch 5/50
2/2 [==============================] - 0s 65ms/step - loss: 750.6425 - mean_squared_error: 750.6425 - val_loss: 285.0132 - val_mean_squared_error: 285.0132
Epoch 6/50
2/2 [==============================] - 0s 68ms/step - loss: 221.8351 - mean_squared_error: 221.8351 - val_loss: 52.2383 - val_mean_squared_

In [8]:
model_test.save('trained_model.h5', save_format='h5')

In [15]:
import h5py
filename = 'trained_model.h5'

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])
    print(data)

Keys: <KeysViewHDF5 ['model_weights', 'optimizer_weights']>
['dense', 'dense_1', 'input_1', 'top_level_model_weights']


In [19]:
data['dense']

TypeError: list indices must be integers or slices, not str

In [20]:
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

f = h5py.File(filename, 'r')
f.visititems(print_attrs)

model_weights
    backend: tensorflow
    keras_version: 2.8.0
    layer_names: ['input_1' 'dense' 'dense_1']
model_weights/dense
    weight_names: ['dense/kernel:0' 'dense/bias:0']
model_weights/dense/dense
model_weights/dense/dense/bias:0
model_weights/dense/dense/kernel:0
model_weights/dense_1
    weight_names: ['dense_1/kernel:0' 'dense_1/bias:0']
model_weights/dense_1/dense_1
model_weights/dense_1/dense_1/bias:0
model_weights/dense_1/dense_1/kernel:0
model_weights/input_1
    weight_names: []
model_weights/top_level_model_weights
    weight_names: []
optimizer_weights
    weight_names: ['Adam/iter:0' 'Adam/dense/kernel/m:0' 'Adam/dense/bias/m:0'
 'Adam/dense_1/kernel/m:0' 'Adam/dense_1/bias/m:0' 'Adam/dense/kernel/v:0'
 'Adam/dense/bias/v:0' 'Adam/dense_1/kernel/v:0' 'Adam/dense_1/bias/v:0']
optimizer_weights/Adam
optimizer_weights/Adam/dense
optimizer_weights/Adam/dense/bias
optimizer_weights/Adam/dense/bias/m:0
optimizer_weights/Adam/dense/bias/v:0
optimizer_weights/Adam/dense/k